In [6]:
# Dependencies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
from census import Census
from us import states
from config import g_key

# Census API Key
from config import census_key

c = Census(census_key, year=2018)
# print(census_key)


In [25]:
# Run Census Search to retrieve data on all zip codes 
# See: https://github.com/CommerceDataService/census-wrapper for library documentation
# See: https://gist.github.com/afhaque/60558290d6efd892351c4b64e5c01e9b for labels
census_data = c.acs5.get(("NAME", "B19013_001E", "B01003_001E", "B01002_001E", "B01002_002E",
                          "B01002_003E","B19301_001E"),
                           {'for': 'state:*'})
                         
census_data
# Convert to DataFrame
census_pd = pd.DataFrame(census_data)

# Column Reordering
census_pd = census_pd.rename(columns={"B01003_001E": "Population",
                                      "B01002_001E": "Median Age",
                                      "B01002_002E": "Median Male Age",
                                      "B01002_003E": "Median Female Age",
                                      "B19013_001E": "Household Income",
                                      "B19301_001E": "Per Capita Income",                                      
                                      "NAME": "Name", "state": "State"})
census_pd.head()                       


,Name,Household Income,Population,Median Age,Median Male Age,Median Female Age,Per Capita Income,State
0,Minnesota,68411.0,5527358.0,37.9,37.0,38.9,36245.0,27
1,Mississippi,43567.0,2988762.0,37.2,35.6,38.6,23434.0,28
2,Missouri,53560.0,6090062.0,38.5,37.1,40.0,29537.0,29
3,Montana,52559.0,1041732.0,39.8,38.7,40.9,29765.0,30
4,Nebraska,59116.0,1904760.0,36.4,35.4,37.4,31101.0,31


In [26]:

# Add in Per Capita Income Rate (Per Capita Income / Population)
census_pd["Per Capita Income Rate"] = 100 * \
    census_pd["Per Capita Income"].astype(
        int) / census_pd["Population"].astype(int)

# Final DataFrame
census_pd = census_pd[["State", "Population", "Median Age", "Median Male Age", "Median Female Age", "Household Income",
                       "Per Capita Income Rate", "Per Capita Income"]]

# Visualize
print(len(census_pd))
census_pd.head()

52


,State,Population,Median Age,Median Male Age,Median Female Age,Household Income,Per Capita Income Rate,Per Capita Income
0,27,5527358.0,37.9,37.0,38.9,68411.0,0.655738,36245.0
1,28,2988762.0,37.2,35.6,38.6,43567.0,0.784070,23434.0
2,29,6090062.0,38.5,37.1,40.0,53560.0,0.485003,29537.0
3,30,1041732.0,39.8,38.7,40.9,52559.0,2.857261,29765.0
4,31,1904760.0,36.4,35.4,37.4,59116.0,1.632804,31101.0


In [18]:
# Save as a csv
# Note to avoid any issues later, use encoding="utf-8"
census_pd.to_csv("census_data_states.csv", encoding="utf-8", index=False)

In [20]:
centroids = pd.read_csv("state_centroids.csv")
centroids.head()

,State,Latitude,Longitude
0,Alabama,32.7794,-86.8287
1,Alaska,64.0685,-152.2782
2,Arizona,34.2744,-111.6602
3,Arkansas,34.8938,-92.4426
4,California,37.1841,-119.4696


In [21]:
# Merge the datasets using the sate columns
census_data = pd.merge(census_pd, centroids, how="left", left_on="Name", right_on="State")

# Save the updated dataframe as a csv
census_data.to_csv("state_census_data.csv", encoding="utf-8", index=False)
census_data.head()

,Name,Household Income,Population,Median Age,Median Male Age,Median Female Age,Per Capita Income,state,State,Latitude,Longitude
0,Minnesota,68411.0,5527358.0,37.9,37.0,38.9,36245.0,27,Minnesota,46.2807,-94.3053
1,Mississippi,43567.0,2988762.0,37.2,35.6,38.6,23434.0,28,Mississippi,32.7364,-89.6678
2,Missouri,53560.0,6090062.0,38.5,37.1,40.0,29537.0,29,Missouri,38.3566,-92.4580
3,Montana,52559.0,1041732.0,39.8,38.7,40.9,29765.0,30,Montana,47.0527,-109.6333
4,Nebraska,59116.0,1904760.0,36.4,35.4,37.4,31101.0,31,Nebraska,41.5378,-99.7951


In [ ]:
# # Convert poverty rate as a list
# # Convert watch rate to list
# poverty_rate = census_data["Poverty Rate"].tolist()

In [ ]:
# Configure gmaps
gmaps.configure(api_key = g_key)

In [ ]:
# # Create a map using state centroid coordinates to set markers
# marker_locations = census_data[['Latitude', 'Longitude']]

# # Create a marker_layer using the poverty list to fill the info box
# fig = gmaps.figure()
# markers = gmaps.marker_layer(marker_locations,
#     info_box_content=[f"Poverty Rate: {rate}" for rate in poverty_rate])
# fig.add_layer(markers)
# fig